<a href="https://colab.research.google.com/github/Achillesy/Fetal_Functional_MRI_Segmentation/blob/master/fmri_vnet_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -qq -y git
!git clone https://github.com/Achillesy/Fetal_Functional_MRI_Segmentation.git


fatal: destination path 'Fetal_Functional_MRI_Segmentation' already exists and is not an empty directory.


In [2]:
import subprocess

!cat Fetal_Functional_MRI_Segmentation/models/fold4_train_metric_vnet_part_* > fold4_train_metric_vnet.pth
file_pth = "fold4_train_metric_vnet.pth"
output = subprocess.check_output(["md5sum", file_pth])
md5 = output.split()[0].decode()
expected_md5 = "cd8284f0e56f21a422b277f3be79ae10"
assert md5 == expected_md5, "MD5 value does not match"


In [3]:
!pip install monai

from monai.config import print_config
print_config()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
MONAI version: 1.1.0
Numpy version: 1.22.4
Pytorch version: 2.0.1+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.0.2
scikit-image version: 0.19.3
Pillow version: 8.4.0
Tensorboard version: 2.12.2
gdown version: 4.6.6
TorchVision version: 0.15.2+cu118
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 1.5.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visi

In [4]:
import torch
from types import SimpleNamespace

cfg = SimpleNamespace(**{})
cfg.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
from monai.networks.nets import VNet

model = VNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    act=("elu", {"inplace": True}),
    dropout_prob=0.5,
    dropout_dim=3,
    bias=False,
).to(cfg.device)
model.load_state_dict(torch.load(file_pth, cfg.device))


VNet(
  (in_tr): InputTransition(
    (act_function): ELU(alpha=1.0, inplace=True)
    (conv_block): Convolution(
      (conv): Conv3d(1, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2), bias=False)
      (adn): ADN(
        (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (down_tr32): DownTransition(
    (down_conv): Conv3d(16, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
    (bn1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_function1): ELU(alpha=1.0, inplace=True)
    (act_function2): ELU(alpha=1.0, inplace=True)
    (ops): Sequential(
      (0): LUConv(
        (act_function): ELU(alpha=1.0, inplace=True)
        (conv_block): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2), bias=False)
          (adn): ADN(
            (N): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [6]:
cfg.device

device(type='cpu')